# I Thought It would be fun to turn the well-know Titanic data set into sql and analyze it

In [59]:
import pandas as pd
from sqlalchemy import create_engine

In [60]:
titanic = pd.read_csv('train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<b>VARIABLE DESCRIPTION</b> <br><br>
PassengerId: is differetn id for each passenger><br>
survival didthe passenger survive 0 = No, 1 = Yes<br>
pclass Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd<br>
sex	Sex	male or female <br>
Age	how old the passenger is in years	 <br>
sibsp # of siblings / spouses aboard the Titanic	 <br>
parch # of parents / children aboard the Titanic	 <br>
ticket Ticket number	 <br>
fare How much the passenger paid for thier ticket	 <br>
cabin Cabin number	 <br>
embarked Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampto <br>

In [61]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [62]:
#dropping Cabin Column because too much data missing
titanic = titanic.drop(columns=['Cabin'])

In [63]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [64]:
engine = create_engine('sqlite:///:memory:', echo=True)
conn = engine.connect()

In [65]:
sqlite_table = "td"
titanic.to_sql(sqlite_table, conn)

2023-08-15 15:59:32,270 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-15 15:59:32,271 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("td")
2023-08-15 15:59:32,272 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-15 15:59:32,272 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("td")
2023-08-15 15:59:32,272 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-15 15:59:32,273 INFO sqlalchemy.engine.Engine 
CREATE TABLE td (
	"index" BIGINT, 
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Embarked" TEXT
)


2023-08-15 15:59:32,273 INFO sqlalchemy.engine.Engine [no key 0.00021s] ()
2023-08-15 15:59:32,274 INFO sqlalchemy.engine.Engine CREATE INDEX ix_td_index ON td ("index")
2023-08-15 15:59:32,274 INFO sqlalchemy.engine.Engine [no key 0.00018s] ()
2023-08-15 15:59:32,278 INFO sqlalchemy.engine.Engine INSERT INTO td ("index", "PassengerId", "

891

In [66]:
sql = '''SELECT * FROM td'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-15 15:59:32,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-15 15:59:32,286 INFO sqlalchemy.engine.Engine SELECT * FROM td
2023-08-15 15:59:32,286 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


# lets see what percentage of people survived the crash.

In [67]:
sql = '''SELECT ROUND(SUM(Survived) * 100.0 / COUNT(Survived),1) as 'Percentage Survived' FROM td'''
df = pd.read_sql_query(sql, conn)
df

2023-08-15 15:59:32,295 INFO sqlalchemy.engine.Engine SELECT ROUND(SUM(Survived) * 100.0 / COUNT(Survived),1) as 'Percentage Survived' FROM td
2023-08-15 15:59:32,296 INFO sqlalchemy.engine.Engine [raw sql] ()


,Percentage Survived
0,38.4


In [68]:
#What percentage survived by age groups
sql = '''SELECT (SELECT COUNT(*) FROM td WHERE Age < 18) as Children,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age < 18) as 'Children Perc Survived',
(SELECT COUNT(*) FROM td WHERE Age > 17 and Age < 65) as Adults,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age > 17 and Age < 65) as 'Adults Perc Survived',
(SELECT COUNT(*) FROM td WHERE Age > 64) as Elderly,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age > 64) as 'Edlerly Perc Survived'
FROM TD 
LIMIT 1''' 
df = pd.read_sql_query(sql, conn)
df

2023-08-15 15:59:32,300 INFO sqlalchemy.engine.Engine SELECT (SELECT COUNT(*) FROM td WHERE Age < 18) as Children,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age < 18) as 'Children Perc Survived',
(SELECT COUNT(*) FROM td WHERE Age > 17 and Age < 65) as Adults,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age > 17 and Age < 65) as 'Adults Perc Survived',
(SELECT COUNT(*) FROM td WHERE Age > 64) as Elderly,
(SELECT ROUND(SUM(Survived) * 100.0 / COUNT(*),1) FROM td WHERE Age > 64) as 'Edlerly Perc Survived'
FROM TD 
LIMIT 1
2023-08-15 15:59:32,301 INFO sqlalchemy.engine.Engine [raw sql] ()


,Children,Children Perc Survived,Adults,Adults Perc Survived,Elderly,Edlerly Perc Survived
0,113,54.0,590,38.6,11,9.1


In [69]:
sql = '''SELECT Avg(Fare) FROM td'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-15 15:59:32,306 INFO sqlalchemy.engine.Engine SELECT Avg(Fare) FROM td
2023-08-15 15:59:32,306 INFO sqlalchemy.engine.Engine [raw sql] ()


,Avg(Fare)
0,32.204208


In [70]:
# Going to find what percentage of people survived by different fare amounts
sql = '''SELECT (SELECT COUNT(*) FROM td WHERE Fare > 32) as 'Rich People',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Fare > 32) as 'Rich Perc Survived',
(SELECT COUNT(*) FROM td WHERE Fare < 32) as 'Poor People',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Fare < 32) as 'Poor Perc Survived' 
FROM td LIMIT 1'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-15 15:59:32,311 INFO sqlalchemy.engine.Engine SELECT (SELECT COUNT(*) FROM td WHERE Fare > 32) as 'Rich People',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Fare > 32) as 'Rich Perc Survived',
(SELECT COUNT(*) FROM td WHERE Fare < 32) as 'Poor People',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Fare < 32) as 'Poor Perc Survived' 
FROM td LIMIT 1
2023-08-15 15:59:32,312 INFO sqlalchemy.engine.Engine [raw sql] ()


,Rich People,Rich Perc Survived,Poor People,Poor Perc Survived
0,211,59.7,680,31.8


In [71]:
#what percentage survived male vs female
sql = '''SELECT (SELECT COUNT(*) FROM td WHERE Sex ='male') as 'Males',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Sex = 'male') as 'Males Perc Survived',
(SELECT COUNT(*) FROM td WHERE Sex = 'female') as 'Females',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Sex= 'female') as 'Females Perc Survived' 
FROM td LIMIT 1'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-15 15:59:32,317 INFO sqlalchemy.engine.Engine SELECT (SELECT COUNT(*) FROM td WHERE Sex ='male') as 'Males',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Sex = 'male') as 'Males Perc Survived',
(SELECT COUNT(*) FROM td WHERE Sex = 'female') as 'Females',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Sex= 'female') as 'Females Perc Survived' 
FROM td LIMIT 1
2023-08-15 15:59:32,317 INFO sqlalchemy.engine.Engine [raw sql] ()


,Males,Males Perc Survived,Females,Females Perc Survived
0,577,18.9,314,74.2


In [72]:
#what percentage survived In Different Pclasses
sql = '''SELECT (SELECT COUNT(*) FROM td WHERE Pclass =1) as 'P1',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =1) as 'P1 Perc Survived',
(SELECT COUNT(*) FROM td WHERE Pclass =2) as 'P2',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =2) as 'P2 Perc Survived',
(SELECT COUNT(*) FROM td WHERE Pclass =3) as 'P3',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =3) as 'P3 Perc Survived' 
FROM td LIMIT 1'''
df = pd.read_sql_query(sql, conn)
df.head()

2023-08-15 15:59:32,323 INFO sqlalchemy.engine.Engine SELECT (SELECT COUNT(*) FROM td WHERE Pclass =1) as 'P1',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =1) as 'P1 Perc Survived',
(SELECT COUNT(*) FROM td WHERE Pclass =2) as 'P2',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =2) as 'P2 Perc Survived',
(SELECT COUNT(*) FROM td WHERE Pclass =3) as 'P3',
(SELECT ROUND(Sum(Survived)*100.0/Count(*),1) From td WHERE Pclass =3) as 'P3 Perc Survived' 
FROM td LIMIT 1
2023-08-15 15:59:32,323 INFO sqlalchemy.engine.Engine [raw sql] ()


,P1,P1 Perc Survived,P2,P2 Perc Survived,P3,P3 Perc Survived
0,216,63.0,184,47.3,491,24.2


In [73]:
conn.close()

2023-08-15 15:59:32,329 INFO sqlalchemy.engine.Engine ROLLBACK
